In [9]:
import pandas as pd
import numpy as np
from pathlib import Path
import ast

# 0) 경로 세팅
ROOT = Path.cwd()
DATA_DIR = ROOT / "dump_vote_ver2"
OUT_DIR = ROOT / "clean_vote_ver2"
OUT_DIR.mkdir(parents=True, exist_ok=True)

csv_path = DATA_DIR / "accounts_user_contacts.csv"

In [10]:
# 1) 로드
df = pd.read_csv(csv_path)

print("shape:", df.shape)
display(df.head())
display(df.dtypes)

shape: (5063, 4)


,id,contacts_count,invite_user_id_list,user_id
0,259,30,[],1167696
1,1756,79,[],863169
2,13742,21,[854615],857205
3,13754,29,[],851431
4,13756,28,[849318],855476


id                      int64
contacts_count          int64
invite_user_id_list    object
user_id                 int64
dtype: object

In [ ]:
# 2) 기본 결측 재확인
summary = pd.DataFrame([{
    "row_cnt": len(df),
    "null_id": df["id"].isna().sum(),
    "null_user_id": df["user_id"].isna().sum(),
    "null_contacts_count": df["contacts_count"].isna().sum(),
    "null_invite_user_id_list": df["invite_user_id_list"].isna().sum(),
}])
display(summary)

,row_cnt,null_id,null_user_id,null_contacts_count,null_invite_user_id_list
0,5063,0,0,0,0


In [ ]:
# 3) contacts_count 타입/이상치 재확인

# 음수 여부
neg_contacts = df[df["contacts_count"] < 0]
print("contacts_count 음수 건수:", len(neg_contacts))
display(neg_contacts.head(20))



contacts_count 음수 건수: 0


,id,contacts_count,invite_user_id_list,user_id


In [ ]:
# 4) invite_user_id_list: 리스트처럼 보이는 문자열 -> 리스트
# "[1,2]" -> [1,2]
# "123" 같은 문자열도 int로

def parse_listlike(x):
    if pd.isna(x):
        return []
    s = str(x).strip()
    if s == "" or s == "[]":
        return []
    try:
        v = ast.literal_eval(s)  
        if isinstance(v, list):
            # 내부 값이 숫자면 int로 정리
            out = []
            for i in v:
                if i is None:
                    continue
                try:
                    out.append(int(i))
                except Exception:
                    pass
            return out
        return []
    except Exception:
        return []

df["invite_user_id_list"] = df["invite_user_id_list"].apply(parse_listlike)

In [16]:
# 변환 확인
display(df[["invite_user_id_list"]].head(20))

,invite_user_id_list
0,[]
1,[]
2,[854615]
3,[]
4,[849318]
5,[855829]
6,"[849318, 849421]"
7,[]
8,"[855626, 856042, 837947]"
9,[]


In [ ]:
# 파싱 결과 길이 분포(잘 반영이 된건지)
df["invite_cnt"] = df["invite_user_id_list"].apply(len)
display(df["invite_cnt"].describe())
# 최대 초대수 10명, 대부분은 초대하지 않은 유저임

count    5063.000000
mean        0.333202
std         0.752303
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max        10.000000
Name: invite_cnt, dtype: float64

In [18]:
type(df.loc[0, "invite_user_id_list"])

list

In [20]:
# 5) 중복 체크 

# 완전 중복 행
dup_cols = ["id", "user_id", "contacts_count"]

dup_all_cnt = df.duplicated(subset=dup_cols, keep=False).sum()
print("완전 동일 행 중복 수:", dup_all_cnt)

# 유저아이디 중복
dup_user_cnt = df.duplicated(subset=["user_id"], keep=False).sum()
print("user_id 중복 행 수:", dup_user_cnt)

# user_id 중복이 있으면 어떤 형태인지 샘플 확인
if dup_user_cnt > 0:
    display(
        df[df.duplicated(subset=["user_id"], keep=False)]
        .sort_values("user_id")
        .head(30)
    )


완전 동일 행 중복 수: 0
user_id 중복 행 수: 0


In [22]:
df.head(20)

,id,contacts_count,invite_user_id_list,user_id,invite_cnt
0,259,30,[],1167696,0
1,1756,79,[],863169,0
2,13742,21,[854615],857205,1
3,13754,29,[],851431,0
4,13756,28,[849318],855476,1
5,13784,31,[855829],1482744,1
6,13798,45,"[849318, 849421]",854615,2
7,13807,28,[],854372,0
8,13815,26,"[855626, 856042, 837947]",858674,3
9,21155,28,[],855526,0


In [ ]:

# -----------------------------
# 6) 최종 컬럼 정리 + 저장
#    - id 유지(요청 반영)
#    - invite_cnt는 분석 편의용: 필요 없으면 저장에서 빼도 됨
# -----------------------------
out_path = OUT_DIR / "accounts_user_contacts_clean.csv"

df_out = df[["id", "user_id", "contacts_count", "invite_user_id_list", "invite_cnt"]].copy()

# 엑셀에서 리스트 컬럼이 보기 불편하면 문자열로 다시 저장하고 싶을 수도 있음
# 그럴 땐 아래처럼 변환해서 저장:
# df_out["invite_user_id_list"] = df_out["invite_user_id_list"].apply(lambda x: str(x))

df_out.to_csv(out_path, index=False, encoding="utf-8-sig")
print("saved:", out_path)
